In [4]:
import os
import pandas as pd
from utils.utils import get_timetsamp_with_random
from utils.map_utils import get_game_info_with_G_eval
from utils.eval_gpt_utils import get_csv,eval_game
import os
import os.path as osp
import networkx as nx
import json
import matplotlib.pyplot as plt
from utils.map_utils import get_game_info_with_G_eval
from utils.map_utils_old import find_all_paths
from utils.clean_utils import compute_hash_for_path
import tiktoken
import openai
import time
import traceback
import pandas as pd
from transformers import LlamaTokenizerFast

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [5]:
result_dir='/share/data/mei-work/kangrui/github/mango/kangrui/data/gpt-games-results-clean-new-new'
map_dir='/share/data/mei-work/kangrui/github/mango/data/'

In [23]:
def get_cut_off_and_walkthrough_text(walkthrough:str,token_size_limit=3600,model_name='gpt-4',cut_off_number=None):
    if model_name.startswith('gpt'):
        encoder = tiktoken.encoding_for_model(model_name)
    elif model_name.startswith("llama"):
        encoder = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")
        
    enc = encoder.encode(walkthrough)
    
    if len(enc) > token_size_limit:
        cut_off_walkthrough_text = encoder.decode(enc[:token_size_limit])
    else:
        cut_off_walkthrough_text = encoder.decode(enc)
        
    #print(int(walkthrough.split('NUM: ')[-2].split('\n')[0]))
    
    if cut_off_number is None:
        cut_off_number = int(cut_off_walkthrough_text.split('NUM: ')[-2].split('\n')[0])
        if cut_off_number > 70:
            cut_off_number = 70
    else:
        cut_off_number=min(int(walkthrough.split('NUM: ')[-2].split('\n')[0]),cut_off_number)

    walkthrough_text = walkthrough.split('NUM: {}'.format(cut_off_number + 1))[0]

    return cut_off_number,len(encoder.encode(walkthrough_text))

def normalized_edit_distance(s1, s2):
    s1 = s1.lower()
    s2 = s2.lower()
    
    s1=s1.split()
    s2=s2.split()
    
    m = len(s1) + 1
    n = len(s2) + 1

    dp = [[0] * n for _ in range(m)]

    for i in range(m):
        dp[i][0] = i

    for j in range(n):
        dp[0][j] = j

    for i in range(1, m):
        for j in range(1, n):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(
                    dp[i - 1][j] + 1,  # deletion
                    dp[i][j - 1] + 1,  # insertion
                    dp[i - 1][j - 1] + 1  # substitution
                )
    
    # Compute the normalized score
    max_len = max(len(s1), len(s2))
    score = 1 - dp[m - 1][n - 1] / max_len
    return score

def get_edegs(G):
    edges=[]
    for edge in G.edges(data=True):
        edges.append(edge)
    return edges
def num_of_locations(G,cut_off_number):
    edges=get_edegs(G)
    locations=set()
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            locations.add(edge[0])
            locations.add(edge[1])
    return len(locations)

def num_of_edges(G,cut_off_number):
    edges=get_edegs(G)
    cnt=0
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            cnt+=1         
    return cnt

def num_of_exp_edges(G,cut_off_number):
    edges=get_edegs(G)
    cnt=0
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number and edge[2]['seen_in_forward']:
            cnt+=1         
    return cnt
def num_of_exp_edges(G,cut_off_number):
    edges=get_edegs(G)
    cnt=0
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number and edge[2]['seen_in_forward']:
            cnt+=1         
    return cnt

def num_of_imp_edges(G,cut_off_number):
    edges=get_edegs(G)
    cnt=0
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number and (not edge[2]['seen_in_forward']):
            cnt+=1         
    return cnt

def ratio_of_conf_locations(G,all2all,cut_off_number):
    edges=get_edegs(G)
    locations=set()
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            locations.add(edge[0])
            locations.add(edge[1])
    locations=list(locations)
    
    score=0
    for i in range(len(locations)):
        current_max_score = float('-inf')
        for j in range(len(locations)):
            if i != j:  # Don't compare an item with itself
                current_score = normalized_edit_distance(locations[i], locations[j])
                if current_score > current_max_score:
                    current_max_score = current_score
        score+=current_max_score
        
    return score/len(locations) if len(locations)>0 else 0

def num_of_conf_locations(G,all2all,cut_off_number):
    edges=get_edegs(G)
    locations=set()
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            locations.add(edge[0])
            locations.add(edge[1])
    locations=list(locations)
    
    score=0
    for i in range(len(locations)):
        current_max_score = float('-inf')
        for j in range(len(locations)):
            if i != j:  # Don't compare an item with itself
                current_score = normalized_edit_distance(locations[i], locations[j])
                if current_score > current_max_score:
                    current_max_score = current_score
        score+=current_max_score
        
    return score
    
def num_of_df_hard(G,all2all,cut_off_number):
    idset=set()
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number and not path['all_steps_seen_in_forward']:
            idset.add(path['id'])
            cnt+=1
    assert len(idset)==cnt
    return cnt

def num_of_df_easy(G,all2all,cut_off_number):
    idset=set()
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number and path['all_steps_seen_in_forward']:
            idset.add(path['id'])
            cnt+=1
    assert len(idset)==cnt
    return cnt

def is_hard(pair,all2all):
    flag=True
    found=False
    for path in all2all:
        if path['src_node']==pair['src_node'] and path['dst_node']==pair['dst_node'] and path['diff_shortest']==0: 
            found=True
            if path['all_steps_seen_in_forward']:
                flag=False
                break
    assert found
    return flag
                
def num_of_rf_hard(G,all_pairs,all2all,cut_off_number):
    idset=set()
    cnt=0
    for pair in all_pairs:
        if min(pair['path_min_cutoffs'])<=cut_off_number and is_hard(pair,all2all):
            idset.add(pair['id'])
            cnt+=1
    assert len(idset)==cnt
    return cnt

def num_of_rf_easy(G,all_pairs,all2all,cut_off_number):
    idset=set()
    cnt=0
    for pair in all_pairs:
        if min(pair['path_min_cutoffs'])<=cut_off_number and not is_hard(pair,all2all):
            idset.add(pair['id'])
            cnt+=1
    assert len(idset)==cnt
    return cnt

def average_length_of_all2all(all2all,cut_off_number):
    length=0
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number:
            length+=path['step_count']
            cnt+=1
    return length/cnt if cnt>0 else 0

def average_length_of_all2all_simple(all2all,cut_off_number):
    length=0
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number and path['all_steps_seen_in_forward']:
            length+=path['step_count']
            cnt+=1
    return length/cnt if cnt>0 else 0

def average_length_of_all2all_hard(all2all,cut_off_number):
    length=0
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number and not path['all_steps_seen_in_forward']:
            length+=path['step_count']
            cnt+=1
    return length/cnt if cnt>0 else 0
        
def average_num_of_imp_edge(all2all,cut_off_number):
    num=0
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number and not path['all_steps_seen_in_forward']:
            cnt+=1
            for edge in path["path_details"]:
                if not edge["seen_in_forward"]:
                    num+=1
                    
    return num/cnt if cnt>0 else 0

def num_of_special_moves(G,cut_off_number):
    
    reverse_dict = {
    "up": "down",
    "down": "up",
    "north": "south",
    "south": "north",
    "east": "west",
    "west": "east",
    "northeast": "southwest",
    "northwest": "southeast",
    "southeast": "northwest",
    "southwest": "northeast"
}
    
    edges=get_edegs(G)
    actions=set()
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            if edge[2]['action'] not in reverse_dict.keys():
                actions.add(edge[2]['action'])
    return len(actions)

def num_of_tokens_per_edge(G,cut_off_number,token_num):
    return token_num/num_of_exp_edges(G,cut_off_number)

def analyze_map(map_dir,game_name,cut_off_number=None,token_size_limit=3600,model_name='gpt-4'):
    G_eval,G,actions,locations,all2all,all_pairs,walkthrough=get_game_info_with_G_eval(map_dir,game_name)
    cut_off_number,token_num=get_cut_off_and_walkthrough_text(walkthrough,cut_off_number=cut_off_number,
                                                             token_size_limit=token_size_limit,model_name=model_name)
    
    result = {
        'num_of_loactions':num_of_locations(G,cut_off_number),
        'num_of_edges':num_of_edges(G,cut_off_number),
        'average_len_of_simple_path':average_length_of_all2all(all2all,cut_off_number),
        'num_of_steps':cut_off_number,
        'num_of_df_easy':num_of_df_easy(G,all2all,cut_off_number),
        'num_of_df_hard':num_of_df_hard(G,all2all,cut_off_number),
        'num_of_rf_easy':num_of_rf_easy(G,all_pairs,all2all,cut_off_number),
        'num_of_rf_hard':num_of_rf_hard(G,all_pairs,all2all,cut_off_number),
    }
    return result

In [24]:
game_name='zork1'
analyze_map(map_dir,game_name,cut_off_number=200)

{'num_of_loactions': 19,
 'num_of_edges': 34,
 'average_len_of_simple_path': 7.156171284634761,
 'num_of_steps': 200,
 'num_of_df_easy': 351,
 'num_of_df_hard': 46,
 'num_of_rf_easy': 279,
 'num_of_rf_hard': 45}

In [67]:
map_dict1={}
for game_name in os.listdir(map_dir):
    map_dict1[game_name]=analyze_map(map_dir,game_name,cut_off_number=10000,token_size_limit=1500,model_name='llama')
map_dict1 = {key: map_dict1[key] for key in sorted(map_dict1.keys())}

Token indices sequence length is longer than the specified maximum sequence length for this model (10476 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5969 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (11041 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (9479 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (11560 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence l

Token indices sequence length is longer than the specified maximum sequence length for this model (31649 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (26204 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (19340 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5971 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5114 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence l

In [68]:
rf_total=0
df_total=0
cnt=0
for k,v in map_dict1.items():
    rf_total+=v['num_of_rf_easy']
    rf_total+=v['num_of_rf_hard']
    df_total+=v['num_of_df_easy']
    df_total+=v['num_of_df_hard']
    cnt+=1

In [69]:
print(rf_total)
print(df_total)
print(cnt)

14698
21046
53


In [70]:
def latex_table_from_dict(data):
    for k,v in data.items():
        for kk,vv in v.items():
            if isinstance(vv,float):
                v[kk]=round(float(vv), 2)
    
    # Extract row and column names from the dictionary
    rows = list(data.keys())
    columns = list(data[rows[0]].keys())

    # Start table construction
    latex_code = """
\\begin{table}[htbp]
\\small
\\setlength{\\tabcolsep}{4pt}
\\begin{sc}
\\begin{subtable}{1.0\\linewidth}
\\begin{center}
\\begin{tabular}{l""" + "r" * len(columns) + """}
\\toprule
Method """ 

    # Column names
    for col in columns:
        latex_code += "& " + col + " "
    latex_code += "\\\\ \n\\midrule\n"

    # Row data
    for row in rows:
        latex_code += row + " "
        for col in columns:
            val = data[row].get(col, "*")  # Extract value or set "*" if not present
            latex_code += "& " + str(val) + " "
        latex_code += "\\\\\n"

    # End table construction
    latex_code += """
\\bottomrule
\\end{tabular}
\\end{center}
\\end{subtable}%
\\end{sc}
\\caption{Your caption here.}
\\label{tab:yourlabelhere}
\\end{table}
    """
    return latex_code

In [71]:
latex_code=latex_table_from_dict(map_dict1)

In [72]:
print(latex_code)


\begin{table}[htbp]
\small
\setlength{\tabcolsep}{4pt}
\begin{sc}
\begin{subtable}{1.0\linewidth}
\begin{center}
\begin{tabular}{lrrrrrrrr}
\toprule
Method & num_of_loactions & num_of_edges & average_len_of_simple_path & num_of_steps & num_of_df_easy & num_of_df_hard & num_of_rf_easy & num_of_rf_hard \\ 
\midrule
905 & 5 & 7 & 1.88 & 21 & 11 & 5 & 11 & 5 \\
advent & 31 & 57 & 7.79 & 276 & 692 & 100 & 532 & 100 \\
adventureland & 18 & 35 & 6.13 & 169 & 579 & 80 & 260 & 46 \\
afflicted & 11 & 20 & 2.95 & 97 & 100 & 10 & 100 & 10 \\
anchor & 25 & 46 & 5.99 & 530 & 327 & 153 & 302 & 132 \\
awaken & 15 & 28 & 5.02 & 56 & 365 & 45 & 171 & 25 \\
balances & 11 & 18 & 3.09 & 121 & 96 & 8 & 76 & 8 \\
ballyhoo & 17 & 35 & 4.83 & 415 & 302 & 188 & 213 & 59 \\
curses & 14 & 27 & 3.62 & 815 & 182 & 13 & 182 & 0 \\
cutthroat & 25 & 49 & 6.66 & 335 & 471 & 362 & 360 & 216 \\
deephome & 27 & 49 & 4.83 & 326 & 429 & 19 & 429 & 19 \\
detective & 32 & 40 & 8.79 & 50 & 505 & 4 & 505 & 4 \\
dragon & 21 & 4